<a href="https://colab.research.google.com/github/Diame58/Pronobo/blob/main/probono.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import librosa
import librosa.display
import resampy

import plotly.express as px
import IPython.display as ipd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from tqdm import tqdm, trange
from librosa import feature, amplitude_to_db, load

from tqdm.auto import tqdm
from plotly.subplots import make_subplots

from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Activation , Dropout

pd.plotting.register_matplotlib_converters()

%matplotlib inline

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
data_path = '/content/drive/MyDrive/UrbanSound8K.csv'
df = pd.read_csv(data_path)
df.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [3]:
sample_path = []
path = '/content/drive/MyDrive/'

for index_num,row in tqdm(df.iterrows()):
    file_name = os.path.join(os.path.abspath(path),'fold'+str(row["fold"])+'/',str(row['slice_file_name']))
    sample_path.append(file_name)

0it [00:00, ?it/s]

In [4]:
df['path'] = sample_path
df

,slice_file_name,fsID,start,end,salience,fold,classID,class,path
0,100032-3-0-0.wav,100032,0.000000,0.317551,1,5,3,dog_bark,/content/drive/MyDrive/fold5/100032-3-0-0.wav
1,100263-2-0-117.wav,100263,58.500000,62.500000,1,5,2,children_playing,/content/drive/MyDrive/fold5/100263-2-0-117.wav
2,100263-2-0-121.wav,100263,60.500000,64.500000,1,5,2,children_playing,/content/drive/MyDrive/fold5/100263-2-0-121.wav
3,100263-2-0-126.wav,100263,63.000000,67.000000,1,5,2,children_playing,/content/drive/MyDrive/fold5/100263-2-0-126.wav
4,100263-2-0-137.wav,100263,68.500000,72.500000,1,5,2,children_playing,/content/drive/MyDrive/fold5/100263-2-0-137.wav
...,...,...,...,...,...,...,...,...,...
8727,99812-1-2-0.wav,99812,159.522205,163.522205,2,7,1,car_horn,/content/drive/MyDrive/fold7/99812-1-2-0.wav
8728,99812-1-3-0.wav,99812,181.142431,183.284976,2,7,1,car_horn,/content/drive/MyDrive/fold7/99812-1-3-0.wav
8729,99812-1-4-0.wav,99812,242.691902,246.197885,2,7,1,car_horn,/content/drive/MyDrive/fold7/99812-1-4-0.wav
8730,99812-1-5-0.wav,99812,253.209850,255.741948,2,7,1,car_horn,/content/drive/MyDrive/fold7/99812-1-5-0.wav


In [5]:
count1 = (df['class'] == "car_horn").sum()
count3 = (df['class'] == "dog_bark").sum()
count8 = (df['class'] == "siren").sum()
print(count1,count3,count8)
#클래스1,3,8인 경우의 데이터 개수

429 1000 929


In [6]:
target_count = count1

car_horn_data = df[df['class'] == "car_horn"]
dog_bark_data = df[df['class'] == "dog_bark"]
siren_data = df[df['class'] == "siren"]

car_horn_sampled = car_horn_data.sample(n=target_count, random_state=42)
dog_bark_sampled = dog_bark_data.sample(n=target_count, random_state=42)
siren_sampled = siren_data.sample(n=target_count, random_state=42)

# 클래스 1의 데이터와 샘플링한 클래스 3과 8의 데이터를 합치기
df1 = pd.concat([car_horn_sampled, dog_bark_sampled, siren_sampled])

# 결과 출력
print(df1)

       slice_file_name    fsID      start        end  salience  fold  classID  \
8724   99812-1-0-1.wav   99812  38.676254  42.676254         2     7        1   
3440   17074-1-0-0.wav   17074   3.540527   4.403714         2    10        1   
5149  199769-1-0-6.wav  199769   3.848273   7.848273         1     3        1   
1311  132073-1-4-0.wav  132073   8.310243   9.094228         2     2        1   
7602   72567-1-4-7.wav   72567  58.180670  62.180670         2     2        1   
...                ...     ...        ...        ...       ...   ...      ...   
5773  24347-8-0-27.wav   24347  17.620371  21.620371         2     4        8   
6409   42117-8-0-0.wav   42117  88.660252  92.660252         2     3        8   
5780  24347-8-0-33.wav   24347  20.620371  24.620371         2     4        8   
2675  159745-8-1-3.wav  159745   3.199906   7.199906         2     9        8   
2475  157866-8-0-6.wav  157866   3.000000   7.000000         1     9        8   

         class             

In [7]:
def features_extract(file_name):
    sample, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
    feature = librosa.feature.mfcc(y=sample, sr=sample_rate, n_mfcc=50)
    scaled_feature = np.mean(feature.T, axis=0)
    return scaled_feature

extracted = []
path = '/content/drive/MyDrive/'

for index_num, row in tqdm(df1.iterrows()):
    file_name = os.path.join(os.path.abspath(path), 'fold' + str(row["fold"]), str(row['slice_file_name']))
    final_class_labels = row['class']
    data = features_extract(file_name)
    extracted.append([data, final_class_labels])

0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1103
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1323
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1523
  warnings.warn(


In [8]:
ext_df = pd.DataFrame(extracted,columns=['feature','class'])
ext_df1=ext_df[ext_df['class'].isin(["car_horn","dog_bark","siren"])]
ext_df1

,feature,class
0,"[-346.73447, 117.378136, -46.578705, 33.662792...",car_horn
1,"[-317.48117, 155.55836, 4.206973, 52.135128, 1...",car_horn
2,"[-170.06764, 118.21796, -28.936142, 19.422232,...",car_horn
3,"[-490.05728, 96.37583, -28.977787, 34.217995, ...",car_horn
4,"[-316.254, 194.07347, -165.19958, 43.953053, -...",car_horn
...,...,...
1282,"[-225.59827, 165.52647, -55.945892, -4.5067225...",siren
1283,"[-220.92384, 166.31999, -13.943059, 23.526903,...",siren
1284,"[-199.61028, 174.74158, -66.42286, 6.6663556, ...",siren
1285,"[-599.5049, 215.67958, -15.634457, -0.04080167...",siren


In [9]:
x = np.array(ext_df1['feature'].tolist())
y = np.array(ext_df1['class'].tolist())

In [10]:
le = LabelEncoder()
y = to_categorical(le.fit_transform(y))

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state = 42)

print("Number of training samples = ", x_train.shape[0])
print("Number of testing samples = ",x_test.shape[0])

Number of training samples =  1029
Number of testing samples =  258


In [12]:
num_labels = y.shape[1]
model = tf.keras.Sequential()

model.add(Dense(128, input_shape=(50,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(128))

model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               6528      
                                                                 
 activation (Activation)     (None, 128)               0         
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 activation_1 (Activation)   (None, 256)               0         
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               6

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.fit(
          x_train,
          y_train,
          batch_size=32,
          epochs=100,
          validation_data=(x_test, y_test),
         )
#dropout을 높일수록 정확도 떨어짐

Epoch 1/100
33/33 [==============================] - 2s 17ms/step - loss: 19.8665 - accuracy: 0.3693 - val_loss: 1.7027 - val_accuracy: 0.5426
Epoch 2/100
33/33 [==============================] - 0s 7ms/step - loss: 7.1237 - accuracy: 0.4198 - val_loss: 0.7806 - val_accuracy: 0.6008
Epoch 3/100
33/33 [==============================] - 0s 7ms/step - loss: 3.6905 - accuracy: 0.4558 - val_loss: 0.8168 - val_accuracy: 0.5310
Epoch 4/100
33/33 [==============================] - 0s 7ms/step - loss: 2.5705 - accuracy: 0.4490 - val_loss: 0.7571 - val_accuracy: 0.6512
Epoch 5/100
33/33 [==============================] - 0s 11ms/step - loss: 1.7550 - accuracy: 0.4781 - val_loss: 0.8163 - val_accuracy: 0.5504
Epoch 6/100
33/33 [==============================] - 0s 11ms/step - loss: 1.6363 - accuracy: 0.4694 - val_loss: 0.8293 - val_accuracy: 0.6085
Epoch 7/100
33/33 [==============================] - 0s 11ms/step - loss: 1.4617 - accuracy: 0.5015 - val_loss: 0.8112 - val_accuracy: 0.6202
Epoch 8/

In [ ]:
model.save('my_model', save_format="tf")

# 모델 로드
model = tf.keras.models.load_model('my_model')
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# 변환된 모델 저장
tflite_model = converter.convert()
tflite_model_path = '/content/drive/MyDrive/my_model.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model_weights.h5")

In [ ]:
from tensorflow.keras.models import model_from_json

with open("model.json", "r") as json_file:
    loaded_model_json = json_file.read()
model = model_from_json(loaded_model_json)
model.load_weights("model_weights.h5")

In [ ]:
h5_model_path = '/content/drive/MyDrive/my_model.h5'
model = load_model(h5_model_path)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

tflite_model_path = '/content/drive/MyDrive/my_model.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

In [ ]:
def extract_feature(file_name):
    audio_data, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
    fea = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=50)
    scaled = np.mean(fea.T,axis=0)
    return np.array([scaled])

In [ ]:
def print_prediction(file_name):
    pred_fea = extract_feature(file_name)
    pred_vector = np.argmax(model.predict(pred_fea), axis=-1)
    pred_class = le.inverse_transform(pred_vector)
    print("The predicted class is:", pred_class[0], '\n')

In [ ]:
print_prediction('/content/drive/MyDrive/fold1/24074-1-0-2.wav')
ipd.Audio('/content/drive/MyDrive/fold1/24074-1-0-2.wav')
#차소리

In [ ]:
print_prediction('/content/drive/MyDrive/fold1/7383-3-1-0.wav')
ipd.Audio('/content/drive/MyDrive/fold1/7383-3-1-0.wav')
#dog_bark

In [ ]:
print_prediction('/content/drive/MyDrive/fold1/30823-8-0-0.wav')
ipd.Audio('/content/drive/MyDrive/fold1/30823-8-0-0.wav')
#siren

In [ ]:
model.save(path='C:\\Users\\sojou\\')

In [ ]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model('C:\\Users\\sojou\\') # saved_model_dir은 텐서플로 모델의 디렉토리 경로
tflite_model = converter.convert()
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)